<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/infer_ort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime
!wget https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
!wget https://github.com/onnx/models/raw/master/vision/classification/squeezenet/model/squeezenet1.1-7.onnx

     |████████████████████████████████| 4.9 MB 5.3 MB/s 
--2022-02-02 12:47:54--  https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
Resolving akm-img-a-in.tosshub.com (akm-img-a-in.tosshub.com)... 23.202.199.54, 2600:1407:d800:194::1ac0, 2600:1407:d800:188::1ac0
Connecting to akm-img-a-in.tosshub.com (akm-img-a-in.tosshub.com)|23.202.199.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30622 (30K) [image/jpeg]
Saving to: ‘jet.jpeg’

jet.jpeg            100%[===================>]  29.90K  95.4KB/s    in 0.3s    

2022-02-02 12:47:57 (95.4 KB/s) - ‘jet.jpeg’ saved [30622/30622]

--2022-02-02 12:47:57--  https://github.com/onnx/models/raw/master/vision/classification/squeezenet/model/squeezenet1.1-7.onnx
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/mast

In [ ]:
import numpy as np    
import onnxruntime    
#import onnx
#from onnx import helper
from PIL import Image
import argparse

model_path = "./squeezenet1.1-7.onnx"
input_file="./jet.jpeg"

def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
        
    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32')
    return norm_img_data

def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def postprocess(result):
    return softmax(np.array(result)).tolist()

def infer_oonx_model():

  session = onnxruntime.InferenceSession(model_path, None)

  # get the name of the first input of the model
  input_name = session.get_inputs()[0].name  
  input_details  = session.get_inputs()
  output_details = session.get_outputs()
  print('Num Outputs  - ', len(output_details))
  for node in output_details: 
    print(node)   
  print('Num Inputs   - ', len(input_details))
  for node in input_details: 
    print(node)   

  image = Image.open(input_file).resize((224, 224))

  print("Image size: ", image.size)
  image_data = np.array(image).transpose(2, 0, 1)
  input_data = preprocess(image_data)

  raw_result = session.run([], {input_name: input_data})
  res = postprocess(raw_result[0])
  res = np.array(res)

  top_k = res.argsort()[-5:][::-1]
  print(top_k)     
                                                                       
infer_oonx_model()


Num Outputs  -  1
NodeArg(name='536', type='tensor(float)', shape=[1, 1000])
Num Inputs   -  1
NodeArg(name='input.1', type='tensor(float)', shape=[1, 3, 224, 224])
Image size:  (224, 224)
[403 895 744 908 657]
